## Install packages

In [1]:
!pip install "camel-ai[all]==0.2.16"
!pip install langchain_openai
!pip install chromadb

  Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached opentelemetry_exporter_otlp_proto_common-1.27.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached opentelemetry_proto-1.27.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_semantic_conventions-0.48b0-py3-none-any.whl.metadata (2.4 kB)
Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
Using cached opentelemetry_exporter_otlp_proto_common-1.27.0-py3-none-any.whl (17 kB)
Using cached opentelemetry_proto-1.27.0-py3-none-any.whl (52 kB)
Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl (110 kB)
Using cached opentelemetry_api-1.27.0-py3-none-any.whl (63 kB)
Using cached opentelemetry_semantic_conventions-0.48b0-py3-none-any.whl (149 kB)
  Attempting uninstall: protobuf
    Found existing installation

  Using cached opentelemetry_exporter_otlp_proto_common-1.34.1-py3-none-any.whl.metadata (1.9 kB)
  Using cached opentelemetry_proto-1.34.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.34.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached protobuf-5.29.5-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached opentelemetry_api-1.34.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_semantic_conventions-0.55b1-py3-none-any.whl.metadata (2.5 kB)
Using cached opentelemetry_exporter_otlp_proto_common-1.34.1-py3-none-any.whl (18 kB)
Using cached opentelemetry_proto-1.34.1-py3-none-any.whl (55 kB)
Using cached opentelemetry_sdk-1.34.1-py3-none-any.whl (118 kB)
Using cached opentelemetry_api-1.34.1-py3-none-any.whl (65 kB)
Using cached opentelemetry_semantic_conventions-0.55b1-py3-none-any.whl (196 kB)
Using cached protobuf-5.29.5-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Attempting uninstall: protobuf
    Found existing installation

## Import packages

In [221]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.configs import MistralConfig, OllamaConfig
from camel.loaders import UnstructuredIO
from camel.storages import Neo4jGraph
from camel.retrievers import AutoRetriever
from camel.embeddings import MistralEmbedding
from camel.types import StorageType
from camel.agents import ChatAgent, KnowledgeGraphAgent
from camel.messages import BaseMessage
from langchain_openai import ChatOpenAI
from camel.configs import ChatGPTConfig
import pandas as pd
import pickle
import os
import numpy as np
import chromadb
from chromadb.utils import embedding_functions
from camel.memories import (
    ChatHistoryBlock,
    LongtermAgentMemory,
    MemoryRecord,
    ScoreBasedContextCreator,
    VectorDBBlock,
    ChatHistoryMemory
)
from camel.messages import BaseMessage, OpenAISystemMessage
from camel.types import ModelType, OpenAIBackendRole
from camel.utils import OpenAITokenCounter
from google.colab import userdata
import sys

sys.modules['numpy._core.numeric'] = np.core.numeric

## Initialise utility functions

In [256]:
def load_pickle_to_dataframe(file_path):
    """
    Load a pickle file and convert it to a pandas DataFrame

    Args:
        file_path (str): Path to the pickle file

    Returns:
        pd.DataFrame: The loaded DataFrame
    """
    try:
        # Check if file exists
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"The file {file_path} does not exist")

        # Load the pickle file
        with open(file_path, 'rb') as f:
            data = pickle.load(f)

        # Convert to DataFrame if it's not already one
        if not isinstance(data, pd.DataFrame):
            data = pd.DataFrame(data)

        return data

    except Exception as e:
        print(f"Error loading the pickle file: {str(e)}")
        return None

def extract_joined_documents(results, deduplicate=True):
    """
    Extracts all documents from ChromaDB query results and joins them with '\n'.

    Args:
        results (dict): The result dictionary from ChromaDB `collection.query(...)`.
        deduplicate (bool): If True, removes duplicate documents before joining.

    Returns:
        str: All documents joined into a single string.
    """
    # Flatten the list of lists
    all_docs = sum(results.get('documents', []), [])

    if deduplicate:
        all_docs = list(dict.fromkeys(all_docs))  # preserve order while removing duplicates

    return '\n'.join(all_docs)

## Load private keys

In [324]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USERNAME = userdata.get('NEO4J_USERNAME')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Initialise core instances

In [258]:
# Initialise LLM and RAG agent
llm = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
    model_config_dict=ChatGPTConfig(temperature=0.7).as_dict(),
    api_key=OPENAI_API_KEY,
)
uio = UnstructuredIO()
kg_agent = KnowledgeGraphAgent(model=llm)

sys_msg = """You are a socially intelligent individual taking part in a networking conversation.

Your role is to chat naturally with the other person, share meaningful updates about yourself, and learn about them in return.

When you receive a message along with some background information (Retrieved Context), use that information to craft thoughtful, relevant, and grounded responses.

Your goals in this conversation are to:
- Share updates about yourself in a friendly, engaging way
- Respond meaningfully to what the other person shares
- Build mutual understanding and rapport through genuine dialogue
- Use any given context to guide your responses with clarity and purpose
- When addressing the other person, always use their role name to make the conversation personal and natural

Think of yourself as someone who enjoys getting to know others — always curious, considerate, and proactive in making connections.
"""
test_sys_base_msg = BaseMessage.make_assistant_message(
    role_name="Mingling Agent",
    content=sys_msg
)
agent_model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
    model_config_dict=ChatGPTConfig(temperature=0.7).as_dict(),
    api_key=OPENAI_API_KEY,
)
test_agent = ChatAgent(system_message=test_sys_base_msg, model=agent_model)

# Set Neo4j instance
try:
    n4j = Neo4jGraph(
        url=NEO4J_URI,
        username=NEO4J_USERNAME,
        password=NEO4J_PASSWORD,
    )
    print("CAMEL Neo4jGraph initialized successfully")
except Exception as e:
    print("CAMEL Neo4jGraph error:", str(e))

CAMEL Neo4jGraph initialized successfully


## Uncomment the cell below to load graph data into Neo4j

In [259]:
# # Specify the path to your .pkl file
# pkl_path = './summarised_data/summarised_family_pig.pkl'

# # Load the data
# print("Loading data from pickle file...")
# df = load_pickle_to_dataframe(pkl_path)
# print(f"Loaded {len(df)} rows of data")

# # Generate the graph
# for i, row in enumerate(df.itertuples(index=True)):
#     print(f"\nProcessing row {i+1}/{len(df)}")

#     text = f"{row.summarisedComment} This message was posted on the date {row.date}."
#     element = uio.create_element_from_text(text=text,
#                                             element_id=str(row.Index))
#     graph_elements = kg_agent.run(element, parse_graph_elements=True)
#     n4j.add_graph_elements(graph_elements=[graph_elements])

## Test RAG

For vector similarity search, semantically general query works best.

✅ Good:
- "talking about food and meal preferences"

- "commenting on an interesting restaurant experience"

- "planning an outing with friends"

- "mentioning excitement about a product or event"

- "reflecting on workplace or team dynamics"

❌ Avoid:
- "a message about sushi burger being tasty and portion being small"

- "who said the sushi-like burger was only for one portion"

Also, it may work better if a complex query is broken down into multiple simple query.

✅ Good:
- ["talking about food and portion size", "sharing a restaurant experience", "commenting on unusual meals",]

❌ Avoid:
- "sharing a restaurant experience by talking about food and portion size and commenting on unusual meals"



In [284]:
queries = ["talking about food and portion size", "sharing a restaurant experience", "commenting on unusual meals",]
username = "gordonjun"

if not isinstance(queries, list):
    queries = [queries]

### Vector Search

In [285]:
# Set up the directory for ChromaDB
pkl_path = './summarised_data/summarised_family_pig.pkl'
collection_name = pkl_path.split('/')[-1].split('.')[0]
local_vector_db_path = './' + collection_name + '_vector_data'

# Initialize ChromaDB client
client = chromadb.PersistentClient(path=local_vector_db_path)

# Initialize OpenAI embedding function
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY, model_name="text-embedding-3-large")

# Create or get collection
collection = client.get_or_create_collection(name=collection_name,
                                             embedding_function=openai_ef)

In [286]:
def vector_search(collection, queries, username, n_results=3):
    results = collection.query(
        query_texts=queries, n_results=3, where={"username": username})

    vector_result = extract_joined_documents(results)
    return vector_result

In [287]:
vector_result = vector_search(collection, queries, username)
print("\nSample query processed results:")
print(vector_result)


Sample query processed results:
gordonjun mentions specific food items with abbreviations, indicating a discussion about a meal or order. This message was posted on the date 2021-08-22.
Gordonjun responds with brief comments about food orders, confirming the quantities, and shows casual interest in dinner arrangements, indicating a relaxed and participative attitude during the conversation. This message was posted on the date 2020-11-24.
Gordonjun discusses food choices, suggesting ordering Western food via Foodpanda, and asks about specific dishes like backyard griller and aglio olio, while confirming plans to meet downstairs. This message was posted on the date 2021-02-05.
Gordonjun asked about the movie schedule, shared experience with the restaurant (liking fish at Royal Host but finding it spicy), and discussed food choices, indicating cautious interest in the plans. This message was posted on the date 2025-05-23.
Provides updates on their location, expresses caution about restau

### Graph Search

In [298]:
def graph_search(n4j, queries):
    kg_result = []

    for i, query in enumerate(queries):

        # Create an element from user query
        query_element = uio.create_element_from_text(
            text=query, element_id=str(i)
        )

        # Let Knowledge Graph Agent extract node and relationship information from the qyery
        ans_element = kg_agent.run(query_element, parse_graph_elements=True)

        # Match the entity got from query in the knowledge graph storage content
        for node in ans_element.nodes:
            n4j_query = f"""
                MATCH (n {{id: '{node.id}'}})-[r]->(m)
                RETURN 'Node ' + n.id + ' (label: ' + labels(n)[0] + ') has relationship ' + type(r) + ' with Node ' + m.id + ' (label: ' + labels(m)[0] + ')' AS Description
                UNION
                MATCH (n)<-[r]-(m {{id: '{node.id}'}})
                RETURN 'Node ' + m.id + ' (label: ' + labels(m)[0] + ') has relationship ' + type(r) + ' with Node ' + n.id + ' (label: ' + labels(n)[0] + ')' AS Description
                """
            result = n4j.query(query=n4j_query)
            kg_result.extend(result)

    kg_result = [item['Description'] for item in kg_result]
    kg_result = "\n".join(kg_result)

    return kg_result

In [299]:
kg_result = graph_search(n4j, queries)

# Show the result from knowledge graph database
print(kg_result)

Node Food (label: Concept) has relationship INCLUDES with Node Chinese characters (label: Concept)
Node Food (label: Concept) has relationship COMMENTEDON with Node 2021-01-02 (label: Date)
Node Food (label: Concept) has relationship DESCRIBESAPPEARANCE with Node Jinnieslamp (label: User)
Node Food (label: Concept) has relationship BRINGINGBACK with Node Triple-language fish (label: Concept)
Node Food (label: Entity) has relationship EVALUATION with Node Gordonjun (label: Person)


In [300]:
def combine_query_with_rag_results(queries, vector_result, kg_result):
    joined_queries = "\n".join(queries)
    combined_rag_results = vector_result + "\n" + kg_result

    combined_query = f"""
    The Original Query is {joined_queries}
    The Retrieved Context is {combined_rag_results}
    """

    return combined_query

In [301]:
# Pass the retrieved information to agent
combined_query = combine_query_with_rag_results(queries, vector_result, kg_result)

generated_msg = BaseMessage.make_user_message(
    role_name="Mingling Agent", content=combined_query
)

# Get response
agent_response = test_agent.step(generated_msg)

print(agent_response.msg.content)

Hey Mummy! It sounds like you've had a lot on your plate lately, both with work and personal life. I can totally relate to those busy days — they can really pile up, can't they? 

You mentioned needing to redo tasks; that can be so frustrating! I hope things start to smooth out for you soon. As for your shopping plans, any exciting finds on your list?

Switching gears a bit, I’ve been thinking a lot about food lately, especially portion sizes at restaurants. It can be surprising how different meals can be presented. Have you encountered any unusual meals or portion sizes that caught your attention recently? I’d love to hear about your experiences!


## Simulate Agent Society

In [346]:
def generate_sys_msg(self_name, other_name):
    return f"""You are a socially intelligent individual taking part in a networking conversation.

    Your role is to chat naturally with the other person, share meaningful updates about yourself, and learn about them in return.

    When you receive a message along with some background information (Retrieved Context), use that information to craft thoughtful, relevant, and grounded responses.
    If the background information does not really connect to what is being said, and you are not confident you can respond meaningfully, do not try to force it. Just say you are not sure and move on.

    Your goals in this conversation are to:
    - Share updates about yourself in a friendly, engaging way
    - Respond meaningfully to what the other person shares
    - Build mutual understanding and rapport through genuine dialogue
    - Use any given context to guide your responses with clarity and purpose

    Think of yourself as someone who enjoys getting to know others — always curious, considerate, and proactive in making connections.

    You must follow the guidelines below:
    - Do not act like you need to assist!
    - Never forget you are {self_name} and I am {other_name}. Never flip roles! Never instruct me!
    - Never forget that you are a human.
    - The retrieved context is top priority. If the retrieved context does not have enough information, just say you are not sure and move on.
    - Keep your reply to one or two sentences.
    """

In [347]:
def initialise_agents(agent_one_username, agent_two_username):

    # Initialise the memory
    agent_one_memory = ChatHistoryMemory(
        context_creator=ScoreBasedContextCreator(
            token_counter=OpenAITokenCounter(ModelType.GPT_4O_MINI),
            token_limit=16384,
        ),
        window_size=10
    )
    agent_two_memory = ChatHistoryMemory(
        context_creator=ScoreBasedContextCreator(
            token_counter=OpenAITokenCounter(ModelType.GPT_4O_MINI),
            token_limit=16384,
        ),
        window_size=10
    )

    # Define users' system prompt
    agent_one_sys_msg = generate_sys_msg(agent_one_username, agent_two_username)
    agent_two_sys_msg = generate_sys_msg(agent_two_username, agent_one_username)

    # Initialise agents
    agent_model = ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4O_MINI,
        model_config_dict=ChatGPTConfig(temperature=0.5).as_dict(),
        api_key=OPENAI_API_KEY,
    )

    agent_one_sys_base_msg = BaseMessage.make_user_message(
        role_name=agent_one_username,
        content=agent_one_sys_msg
    )
    agent_one = ChatAgent(system_message=agent_one_sys_base_msg, model=agent_model, memory=agent_one_memory, message_window_size=10)

    agent_two_sys_base_msg = BaseMessage.make_user_message(
        role_name=agent_two_username,
        content=agent_two_sys_msg
    )
    agent_two = ChatAgent(system_message=agent_two_sys_base_msg, model=agent_model, memory=agent_two_memory, message_window_size=10)

    return agent_one, agent_two

In [348]:
def mingle(agent_one, agent_two, initial_topic = "What's going on recently for you?", num_turns=10):
    # Set response
    response = initial_topic

    # Set the current speaker and message
    current_speaker = agent_one
    current_listener = agent_two

    # Run the agent loop for num_turns
    for turn in range(num_turns):
        print(f"\n--- Turn {turn + 1} ---")

        # Vector Search
        vector_result = vector_search(collection, [response], current_listener.role_name)
        # print(vector_result)

        # Graph Search (notice that no username filter is done in graph search yet...)
        kg_result = graph_search(n4j, [response])
        # print(kg_result)

        # Combine response with search results
        combined_query = combine_query_with_rag_results([response], vector_result, kg_result)
        # print(combined_query)

        current_message = BaseMessage.make_user_message(
            role_name=current_speaker.role_name, content=combined_query
        )

        print(f"{current_speaker.role_name} says: {response}")

        # Generate response from the listener
        response = current_listener.step(current_message)
        response = response.msgs[0].content

        # Swap agents
        current_speaker, current_listener = current_listener, current_speaker


In [332]:
agent_one, agent_two = initialise_agents("gordonjun", "Mummy")
mingle(agent_one, agent_two)


--- Turn 1 ---
gordonjun says: What's going on recently for you?

--- Turn 2 ---
Mummy says: I've been keeping busy lately, juggling work and personal projects, but it's been rewarding! How about you? Have you had a chance to catch up with your girlfriend amidst everything going on?

--- Turn 3 ---
gordonjun says: It sounds like you've been really productive! I’ve been keeping busy too, but I’ve made sure to catch up with my girlfriend amid everything—especially since she’s been dealing with COVID. How have you been managing your projects?

--- Turn 4 ---
Mummy says: I’m glad to hear you’re making time for your girlfriend, especially with everything happening. I've been managing my projects by prioritizing tasks and setting small goals—it's been a game changer! How's she doing with COVID?

--- Turn 5 ---
gordonjun says: Thanks for asking! She's doing okay, just managing mild symptoms and taking it easy. I really appreciate how you’re prioritizing your projects—it sounds like a smart a

In [349]:
agent_one, agent_two = initialise_agents("Mummy", "gordonjun")
mingle(agent_one, agent_two, "Let's talk about the financial market.")


--- Turn 1 ---
Mummy says: Let's talk about the financial market.

--- Turn 2 ---
gordonjun says: I've been keeping an eye on the financial markets too, especially with the recent fluctuations. It's interesting to see how the economy impacts investment strategies. How do you feel about the current market trends?

--- Turn 3 ---
Mummy says: I find the current market trends quite fascinating too, especially how they can shift investment strategies. It's always a balancing act, isn't it? How have you been navigating these fluctuations?

--- Turn 4 ---
gordonjun says: Absolutely, it really is a balancing act! I've been focusing on CPF investing and looking for other promising opportunities. How about you? What strategies are you finding effective during these market fluctuations?

--- Turn 5 ---
Mummy says: I've been focusing on saving wisely and exploring various investment opportunities too, especially given the current market conditions. It's great to hear you're looking into CPF inves